In [21]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import xgboost as xgb
import copy
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
warnings.filterwarnings('ignore')#忽略一些警告

# 读取数据

In [22]:
data=pd.read_csv("data/onehot_feature.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196499 entries, 0 to 196498
Data columns (total 41 columns):
Unnamed: 0    196499 non-null int64
时间            196499 non-null int64
小区名           196499 non-null int64
小区房屋出租数量      196499 non-null float64
楼层            196499 non-null int64
总楼层           196499 non-null float64
房屋面积          196499 non-null float64
房屋朝向          196499 non-null object
居住状态          196499 non-null float64
卧室数量          196499 non-null int64
厅的数量          196499 non-null int64
卫的数量          196499 non-null int64
出租方式          196499 non-null float64
区             196499 non-null float64
位置            196499 non-null float64
地铁线路          196499 non-null float64
地铁站点          196499 non-null float64
距离            196499 non-null float64
装修情况          196499 non-null float64
月租金           196499 non-null float64
log_rent      196499 non-null float64
新朝向           196499 non-null object
房+卫+厅         196499 non-null int64
房/总           196499 non-null flo

In [23]:
#将离散特征转换成字符串类型
colunms = ['时间', '新小区名', '居住状态', '出租方式', '区','位置','地铁线路','地铁站点','装修情况','户型','聚类特征']
for col in colunms:
    data[col] = data[col].astype(str)

## 获取x和y

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196499 entries, 0 to 196498
Data columns (total 41 columns):
Unnamed: 0    196499 non-null int64
时间            196499 non-null object
小区名           196499 non-null int64
小区房屋出租数量      196499 non-null float64
楼层            196499 non-null int64
总楼层           196499 non-null float64
房屋面积          196499 non-null float64
房屋朝向          196499 non-null object
居住状态          196499 non-null object
卧室数量          196499 non-null int64
厅的数量          196499 non-null int64
卫的数量          196499 non-null int64
出租方式          196499 non-null object
区             196499 non-null object
位置            196499 non-null object
地铁线路          196499 non-null object
地铁站点          196499 non-null object
距离            196499 non-null float64
装修情况          196499 non-null object
月租金           196499 non-null float64
log_rent      196499 non-null float64
新朝向           196499 non-null object
房+卫+厅         196499 non-null int64
房/总           196499 non-null float64
卫

In [25]:
x_columns=['时间', '新小区名', '小区房屋出租数量', '楼层', '总楼层', '房屋面积','居住状态', '卧室数量',
       '厅的数量', '卫的数量', '出租方式', '区', '位置', '地铁线路', '地铁站点', '距离', '装修情况', 
       '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型','平均值特征1',
       '平均值特征2','有地铁','小区线路数','位置线路数','小区条数大于100','小区平均值特征','朝向平均值特征',
           '站点平均值特征','位置平均值特征']
y_label='log_rent'
x=data[x_columns]
y=data[y_label]
y.isnull().sum()

0

# 构建训练函数

In [26]:
def feature_transformer(x,y,test_size=0.3,random_state=12):
    """
    负责分割数据集并转换onehot特征，返回转换后的稀疏矩阵和特征名
    """
    #1.重新命名列名
    #原列名
    cols=['时间', '新小区名', '小区房屋出租数量', '楼层', '总楼层', '房屋面积','居住状态', '卧室数量',
       '厅的数量', '卫的数量', '出租方式', '区', '位置', '地铁线路', '地铁站点', '距离', '装修情况', 
       '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型','平均值特征1',
       '平均值特征2','有地铁','小区线路数','位置线路数','小区条数大于100','小区平均值特征','朝向平均值特征',
           '站点平均值特征','位置平均值特征']
    #新列名
    new_cols=[chr(65+s)+str(i) for s in range(len(cols)//10+1) for i in range(10)]
    new_cols=new_cols[:len(cols)]
    #特征名映射字典
    cols_map={k:v for k,v in zip(cols,new_ cols)}
    #重新命名列
    x.columns=[cols_map[k] for k in x.columns]
    
    #2.分割数据集
    train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=test_size,random_state=random_state)
    
    #3.转换onehot特征
    #返回稀疏矩阵，有两个优点：
    #1.占用内存大幅减小，让并行成为可能，不然并行的话，内存爆掉
    #2.可以加速xgboost训练
    vector=DictVectorizer(sparse=True)
    x_train=vector.fit_transform(train_x.to_dict(orient='records'))
    x_test=vector.transform(test_x.to_dict(orient='records'))
    features=vector.get_feature_names()
    
    #4.构建原始特征对应的新下标字典  
    #原始特征名
    
    #离散列特征下标字典
    feature_map={k:[] for k in cols}
    for i in range(len(features)):
        for col in cols:
            if features[i].startswith(cols_map[col]):#如果新特征以老特征开头
                feature_map[col].append(i)
                break
    return (x_train,x_test,train_y,test_y),feature_map,cols_map


def train(cols,data,feature_map,num_round = 500):
    """
    负责完成一次xgboost训练，返回测试集rmse
    """
    #1.获取数据集
    train_x,test_x,train_y,test_y=data
    #获取原始特征对应的新特征下标
    index=[]
    for col in cols:
        index.extend(feature_map[col])
    x_train=train_x[:,index]
    x_test=test_x[:,index]
    #3.构建数据格式
    #构建DMatrix数据，可以有效利用硬盘缓存，减少内存占用
    dtrain = xgb.DMatrix(x_train,train_y)
    dtest = xgb.DMatrix(x_test,test_y)
    
    #4.设置训练参数
    param = {'max_depth':5, 
             'eta':0.01, 
             'verbosity':1, 
             'objective':'reg:linear',
             'silent': 1,
             'gamma': 0.01,
             'min_child_weight': 1,
             
            }
    #5.模型训练
    bst = xgb.train(param, dtrain, num_round)
    
    #6.模型预测
    preds = bst.predict(dtest)
    preds=np.exp(preds)-1#转换成真实的租金
    y_true=np.exp(test_y)-1
    
    #7.模型评估
    return np.sqrt(mean_squared_error(y_true,preds))
    
    

In [27]:
#完成特征分割和转换
d,f_map,c_map=feature_transformer(x,y)

# 模型特征筛选

In [26]:
# #构造列名类
# class ColData:
#     cols=np.array(['时间', '小区名', '小区房屋出租数量', '楼层', '总楼层', '房屋面积','居住状态', '卧室数量',
#        '厅的数量', '卫的数量', '出租方式', '区', '位置', '地铁线路', '地铁站点', '距离', '装修情况', 
#        '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型','平均值特征1',
#        '有地铁','聚类特征','小区线路数','位置线路数'])
#     def __init__(self,ids):
#         self.ids=ids
#     def include_names(self):
#         print(type(ColData.cols))
#         return list(ColData.cols[self.ids])
#     def exclude_names(self):
#         non_ids=list(set(range(ColData.cols.shape[0]))-set(self.ids))
#         return list(ColData.cols[non_ids])

In [28]:
#构造筛选特征函数
def select_features(cols,min_score):
    include_features=cols
    exclude_features=[]
    cols=np.array(cols)
    for i in range(cols.shape[0]):
        #选中的特征
        features=list(cols[list(set(range(cols.shape[0]))-set([i]))])
        print("开始第{}次训练:".format(i))
        print("未选中特征：",cols[i])
        rmse=train(features,d,f_map,2500)
        print("开始第{}次训练测试集成绩{}：".format(i,rmse))
        print("-"*10)
        if rmse<=min_score:
            exclude_features=cols[i]
            include_features=features
            min_score=rmse
    return min_score,include_features,exclude_features  

In [29]:
min_score=train(x_columns,d,f_map)



In [30]:
features=x_columns
min_score

2.627438015769018

In [31]:
i=1
while len(features)>0:
    print("开始第{}轮筛选:\n".format(i))
    ms,include_f,exclude_f=select_features(features,min_score)
    if len(exclude_f)<=0:
        break
    features=include_f
    min_score=ms
    print("第{}次筛选成绩:{}\n".format(i,min_score))
    i=i+1
    print("保留特征：",include_f)
    print("排除特征：",exclude_f)
    print("\n*************************\n")
    
print("最终特征：",features)
print("最好成绩：",min_score)

开始第1轮筛选:

开始第0次训练:
未选中特征： 时间
开始第0次训练测试集成绩2.3425739100557874：
----------
开始第1次训练:
未选中特征： 新小区名
开始第1次训练测试集成绩2.3159098956260884：
----------
开始第2次训练:
未选中特征： 小区房屋出租数量
开始第2次训练测试集成绩2.3610726419651975：
----------
开始第3次训练:
未选中特征： 楼层
开始第3次训练测试集成绩2.3482224666430156：
----------
开始第4次训练:
未选中特征： 总楼层
开始第4次训练测试集成绩2.3548737824509476：
----------
开始第5次训练:
未选中特征： 房屋面积
开始第5次训练测试集成绩2.3528631046347734：
----------
开始第6次训练:
未选中特征： 居住状态
开始第6次训练测试集成绩2.343644368107605：
----------
开始第7次训练:
未选中特征： 卧室数量
开始第7次训练测试集成绩2.3447264002503947：
----------
开始第8次训练:
未选中特征： 厅的数量
开始第8次训练测试集成绩2.3488349025371353：
----------
开始第9次训练:
未选中特征： 卫的数量
开始第9次训练测试集成绩2.3455068228561444：
----------
开始第10次训练:
未选中特征： 出租方式
开始第10次训练测试集成绩2.347907742954758：
----------
开始第11次训练:
未选中特征： 区
开始第11次训练测试集成绩2.3470549409629813：
----------
开始第12次训练:
未选中特征： 位置
开始第12次训练测试集成绩2.3442351978396916：
----------
开始第13次训练:
未选中特征： 地铁线路
开始第13次训练测试集成绩2.3457640963554747：
----------
开始第14次训练:
未选中特征： 地铁站点
开始第14次训练测试集成绩2.3514284469336877：
----------
开始第15次训练:
未选中特征： 距离
开始第15次训练

KeyboardInterrupt: 

In [21]:
cols=['小区房屋出租数量','房屋面积', '居住状态', '出租方式', '位置', '地铁站点', '距离', '装修情况', '新朝向', '房+卫+厅', '房/总', '卫/总',  '卧室面积', '楼层比', '平均值特征1', '平均值特征2', '小区线路数', '位置线路数','小区条数大于100']
train(cols,d,f_map)

1.6355982447085897

# 参数搜索

## 构建交叉验证和参数搜索函数

In [32]:
from sklearn.model_selection import KFold
#构建交叉验证函数
def train_cv(data,target,params,num_round=200,k_fold=5,silent=0):
    """
    负责完成一种参数组合的情况下k_flod折交叉验证的平均rmse值
    """
    rmses=[]
    #数据分割
    kfold= KFold(n_splits=k_fold,random_state =None,shuffle=True)
    for i,(train_index,val_index) in zip(range(k_fold),kfold.split(data,target)):
        train_x,val_x,train_y,val_y=data[train_index,:],data[val_index,:],target[train_index],target[val_index]
        #构建DMatrix数据
        dtrain = xgb.DMatrix(train_x,train_y)
        dtest = xgb.DMatrix(val_x,val_y)
        if silent==0:
            print("开始第{}/{}折验证：".format(i,k_fold))
        #模型训练
        bst = xgb.train(params, dtrain, num_round)
        
        #6.模型预测
        preds = bst.predict(dtest)
        preds=np.exp(preds)-1#转换成真实的租金
        y_true=np.exp(val_y)-1
        rmse=np.sqrt(mean_squared_error(y_true,preds))
        if silent==0:
            print("第{}/{}折验证rmse：{}".format(i,k_fold,rmse))
        rmses.append(rmse)
    return sum(rmses)/k_fold

def search_params(x,y,params_grid,n_estimators=200,cv=3,silent=0):
    min_rmse=9999
    best_params=None
    params_list=[[]]
    
    #根据参数表格构建所有参数组合
    for k,v in params_grid.items():
        if isinstance(v,list):
            temp=params_list
            params_list=[i+[j] for j in v for i in temp]
        else:
            params_list=[i+[v] for i in params_list]
    params_list=[{k:v for k,v in zip(params_grid.keys(),v_list)} for v_list in params_list]
    
    for i,params in zip(range(len(params_list)),params_list):
        if silent==0:
            print("开始实验第{}组参数：".format(i),params)
        rmse=train_cv(data=x,target=y,params=params,num_round=n_estimators,k_fold=cv)
        if silent==0:
            print("第{}组参数平均rmse：{}".format(i,rmse))
            print("-"*50)
        if rmse<min_rmse:
            min_rmse=rmse
            best_params=params
    return min_rmse,best_params
              

## 开始搜索


In [34]:
params_dict={
    "objective":'reg:linear',
    'eta':[0.01,0.1,0.5],
    'gamma': [0.01,0.05,0.1],
    'silent': 1,
    'max_depth':[15,25,35], 
    'min_child_weight':[0.5,1,3],
}
cols=['小区房屋出租数量', '楼层', '总楼层', '房屋面积','居住状态', '卧室数量',
       '卫的数量',  '位置',  '地铁站点', '距离', '装修情况', 
       '新朝向', '房+卫+厅', '房/总', '卫/总', '厅/总', '卧室面积', '楼层比', '户型','平均值特征1',
       '平均值特征2','有地铁','小区线路数','位置线路数','小区条数大于100','小区平均值特征','朝向平均值特征',
           '站点平均值特征','位置平均值特征']
#1.获取数据集
train_x,test_x,train_y,test_y=d
#获取原始特征对应的新特征下标
index=[]
for col in cols:
    index.extend(f_map[col])
x_train=train_x[:50000,index]#只用前50000条数据做运算
x_test=test_x[:,index]
#由于要用新下标访问，所以要重置索引
train_y=train_y.reset_index(drop=True)[:50000]
test_y=test_y.reset_index(drop=True)
search_params(x=x_train,y=train_y,params_grid=params_dict,n_estimators=1000)

开始实验第0组参数： {'objective': 'reg:linear', 'eta': 0.01, 'gamma': 0.01, 'silent': 1, 'max_depth': 15, 'min_child_weight': 0.5}
开始第0/3折验证：
第0/3折验证rmse：2.1063645180292268
开始第1/3折验证：
第1/3折验证rmse：2.270438488734099
开始第2/3折验证：
第2/3折验证rmse：2.140966941345831
第0组参数平均rmse：2.172589982703052
--------------------------------------------------
开始实验第1组参数： {'objective': 'reg:linear', 'eta': 0.1, 'gamma': 0.01, 'silent': 1, 'max_depth': 15, 'min_child_weight': 0.5}
开始第0/3折验证：
第0/3折验证rmse：2.1029110307375034
开始第1/3折验证：
第1/3折验证rmse：2.151089425447841
开始第2/3折验证：
第2/3折验证rmse：2.2555141001168315
第1组参数平均rmse：2.169838185434059
--------------------------------------------------
开始实验第2组参数： {'objective': 'reg:linear', 'eta': 0.5, 'gamma': 0.01, 'silent': 1, 'max_depth': 15, 'min_child_weight': 0.5}
开始第0/3折验证：
第0/3折验证rmse：2.4535784391141995
开始第1/3折验证：
第1/3折验证rmse：2.424286846886855
开始第2/3折验证：
第2/3折验证rmse：2.1266645639199573
第2组参数平均rmse：2.334843283307004
--------------------------------------------------
开始实验第3组参数： {'object

开始第0/3折验证：
第0/3折验证rmse：2.306825279691586
开始第1/3折验证：
第1/3折验证rmse：2.244977467458515
开始第2/3折验证：
第2/3折验证rmse：2.1922851419755007
第25组参数平均rmse：2.2480292963752
--------------------------------------------------
开始实验第26组参数： {'objective': 'reg:linear', 'eta': 0.5, 'gamma': 0.1, 'silent': 1, 'max_depth': 35, 'min_child_weight': 0.5}
开始第0/3折验证：
第0/3折验证rmse：2.3549137991155114
开始第1/3折验证：
第1/3折验证rmse：2.592401863979216
开始第2/3折验证：
第2/3折验证rmse：2.29801882782804
第26组参数平均rmse：2.4151114969742555
--------------------------------------------------
开始实验第27组参数： {'objective': 'reg:linear', 'eta': 0.01, 'gamma': 0.01, 'silent': 1, 'max_depth': 15, 'min_child_weight': 1}
开始第0/3折验证：
第0/3折验证rmse：2.171764054695917
开始第1/3折验证：
第1/3折验证rmse：2.2142377501924937
开始第2/3折验证：
第2/3折验证rmse：2.284722624617894
第27组参数平均rmse：2.223574809835435
--------------------------------------------------
开始实验第28组参数： {'objective': 'reg:linear', 'eta': 0.1, 'gamma': 0.01, 'silent': 1, 'max_depth': 15, 'min_child_weight': 1}
开始第0/3折验证：
第0/3折验证rmse

开始第1/3折验证：
第1/3折验证rmse：2.45578081917117
开始第2/3折验证：
第2/3折验证rmse：2.2108615863413847
第50组参数平均rmse：2.3692516599950317
--------------------------------------------------
开始实验第51组参数： {'objective': 'reg:linear', 'eta': 0.01, 'gamma': 0.1, 'silent': 1, 'max_depth': 35, 'min_child_weight': 1}
开始第0/3折验证：
第0/3折验证rmse：2.3555601196130684
开始第1/3折验证：
第1/3折验证rmse：2.1563837142863638
开始第2/3折验证：
第2/3折验证rmse：2.2789944615773194
第51组参数平均rmse：2.2636460984922504
--------------------------------------------------
开始实验第52组参数： {'objective': 'reg:linear', 'eta': 0.1, 'gamma': 0.1, 'silent': 1, 'max_depth': 35, 'min_child_weight': 1}
开始第0/3折验证：
第0/3折验证rmse：2.2950794012987616
开始第1/3折验证：
第1/3折验证rmse：2.2213071464839884
开始第2/3折验证：
第2/3折验证rmse：2.169220925172176
第52组参数平均rmse：2.2285358243183087
--------------------------------------------------
开始实验第53组参数： {'objective': 'reg:linear', 'eta': 0.5, 'gamma': 0.1, 'silent': 1, 'max_depth': 35, 'min_child_weight': 1}
开始第0/3折验证：
第0/3折验证rmse：2.3781817980568287
开始第1/3折验证：
第1/3折验证

开始第2/3折验证：
第2/3折验证rmse：2.187687495347084
第75组参数平均rmse：2.151854122790373
--------------------------------------------------
开始实验第76组参数： {'objective': 'reg:linear', 'eta': 0.1, 'gamma': 0.05, 'silent': 1, 'max_depth': 35, 'min_child_weight': 3}
开始第0/3折验证：
第0/3折验证rmse：2.2177760911023863
开始第1/3折验证：
第1/3折验证rmse：2.06291663889677
开始第2/3折验证：
第2/3折验证rmse：2.2830701173735686
第76组参数平均rmse：2.187920949124242
--------------------------------------------------
开始实验第77组参数： {'objective': 'reg:linear', 'eta': 0.5, 'gamma': 0.05, 'silent': 1, 'max_depth': 35, 'min_child_weight': 3}
开始第0/3折验证：
第0/3折验证rmse：2.3170419334417427
开始第1/3折验证：
第1/3折验证rmse：2.366394134764698
开始第2/3折验证：
第2/3折验证rmse：2.3975321343351474
第77组参数平均rmse：2.3603227341805293
--------------------------------------------------
开始实验第78组参数： {'objective': 'reg:linear', 'eta': 0.01, 'gamma': 0.1, 'silent': 1, 'max_depth': 35, 'min_child_weight': 3}
开始第0/3折验证：
第0/3折验证rmse：2.1943993462539413
开始第1/3折验证：
第1/3折验证rmse：2.389702628638741
开始第2/3折验证：
第2/3折验证rm

(2.141382087254338,
 {'objective': 'reg:linear',
  'eta': 0.01,
  'gamma': 0.05,
  'silent': 1,
  'max_depth': 25,
  'min_child_weight': 0.5})

In [35]:
#1.获取数据集
train_x,test_x,train_y,test_y=d
#获取原始特征对应的新特征下标
index=[]
for col in cols:
    index.extend(f_map[col])
x_train=train_x[:,index]
x_test=test_x[:,index]
#由于要用新下标访问，所以要重置索引
train_y=train_y.reset_index(drop=True)
test_y=test_y.reset_index(drop=True)

## 利用搜索的参数训练模型

In [36]:
# xgb_model = xgb.XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=500,verbosity=1, objective='reg:linear',random_state=12,)
# xgb_model.fit(new_train_x, train_y, early_stopping_rounds=10, eval_metric="rmse",
#         eval_set=[(new_test_x, test_y)])

params={
    "objective":'reg:linear',
    'eta':0.01,
    'gamma': 0.05,
    'silent': 1,
    'max_depth':25, 
    'min_child_weight':0.5,
    'sub_sample':0.6,
    'reg_alpha':0.5,
    'reg_lambda':0.8,
    'colsample_bytree':0.5
}
dtrain = xgb.DMatrix(x_train,train_y)
dtest = xgb.DMatrix(x_test,test_y)
bst = xgb.train(params, dtrain, num_boost_round=1500)
        
#6.模型预测
preds = bst.predict(dtest)
preds=np.exp(preds)-1#转换成真实的租金
y_true=np.exp(test_y)-1
rmse=np.sqrt(mean_squared_error(y_true,preds))
rmse

1.4322241873197603

In [124]:
# cv_params = {'n_estimators': [400, 500, 600, 700, 800]}
# other_params = {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
#                 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

# model = xgb.XGBRegressor(**other_params)
# optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1)
# optimized_GBM.fit(new_train_x, train_y)
# evalute_result = optimized_GBM.grid_scores_
# print('每轮迭代运行结果:{0}'.format(evalute_result))
# print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
# print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
rmse

1.5180160328127699